In [ ]:
import datetime
import requests
import json


response = requests.get('https://ofertahoraria.fi.uba.ar/api/last_update').json()

DATA = {
    "cuatrimestre": f"{response['currentQuarter']['year']}C{response['currentQuarter']['quarter']}",
    "timestamp": f"{response['lastUpdate']}",
}
DATA

In [ ]:
with open('src/data/horarios.js', 'r') as f:
    f.readline() # we remove the first `export const = ` line
    old_json = json.load(f)

old_data = {
            'cuatrimestre': old_json['cuatrimestre'],
            'timestamp': old_json['timestamp']
}

if (DATA is old_data): 
    # Nothing to update!
    exit()

In [ ]:
# Funcion para cuando ofertahoraria flashea y devuelve cualquier cosa
# r = requests.get(f'https://ofertahoraria.fi.uba.ar/api/...').json()
# response = decode_response(r)

def decode_response(dic):
    new_dic = {}
    for k,v in dic.items():        
        if isinstance(v, list):
            new_dic[k] = [decode_response(vv) for vv in v]
        elif isinstance(v, dict):
            new_dic[k] = decode_response(v)
            print(new_dic[k])
        else:
            new_dic[k] = str(v).encode('iso-8859-1', 'strict').decode("utf8", 'ignore')
    return new_dic

In [ ]:
response = requests.get('https://ofertahoraria.fi.uba.ar/api/departamentos').json()
departamentos = response.keys()
departamentos

In [ ]:
materias = []
for d in departamentos:
    response = requests.get(f'https://ofertahoraria.fi.uba.ar/api/departamento/{d}').json()
    if (response): 
        materias.extend(response.keys())
materias

In [ ]:
DATA["materias"] = []
DATA["cursos"] = []

DIAS = ["Domingo", "Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado"]

def format_clases(timetable):
    clases = []
    for clase in timetable:
        if clase["day"] == "Otro": continue
        clases.append({"dia": DIAS.index(clase["day"]),
                       "inicio": clase["from"],
                       "fin": clase["to"]})
    return sorted(clases, key=lambda x: x['dia'])

for m in materias:
    response = requests.get(f'https://ofertahoraria.fi.uba.ar/api/materia/{m}').json()
    if (response):
        comisiones = []
        for c in response['commisions']:
            clases = format_clases(c["timetable"])
            if not clases: continue
            comisiones.append({"codigo": f"{m}-{c['name']}", 
                               "materia": m,
                               "docentes": c["teachers"],
                               "clases": clases})
        
        if not comisiones: continue
        
        DATA["materias"].append({"codigo": m,
                                 "nombre": response["description"],
                                 "cursos": sorted(list(map(lambda x: x["codigo"], comisiones)))})
        
        DATA["cursos"].extend(comisiones)
DATA

In [ ]:
DATA['materias'] = sorted(DATA['materias'], key=lambda x: x['codigo'])
DATA['cursos'] = sorted(DATA['cursos'], key=lambda x: x['codigo'])

dump = json.dumps(DATA, indent=2, ensure_ascii=False, sort_keys=True)

with open('src/data/horarios.js', 'w') as fw:
    fw.write("export const data = ")
    fw.write("\n") # Do not remove me. Make me easy to parse.
    fw.write(dump)    